In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from sklearn import linear_model

import holoviews as hv; hv.extension('bokeh', logo=False)
import panel as pn; pn.extension()
import hvplot.pandas

In [ ]:
n = 100
L = 4
x = np.linspace(0,L,n)
f = np.power(x,2) # Parabola with 100 data points

M = 20 # Polynomial degree

phi = np.zeros((n,M))
for j in range(M):
    phi[:,j] = np.power(x,j) # Build matrix A

imgs = []
for j in range(4):
    fn  = np.power(x,2) + 0.1*np.random.randn(*x.shape)
    an  = np.linalg.pinv(phi) @ fn # Least-square fit
    fna = phi @ an 
    En  = np.linalg.norm(f-fna,ord=2)/np.linalg.norm(f,ord=2)
    imgs.append( hv.Spikes( (range(M),an), kdims="Index", vdims="y_fit") )

pn.Column( hv.Curve( (x,f)).opts(width=400),
          hv.Layout( imgs).opts(hv.opts.Spikes(height=200)).cols(2)
         )

In [ ]:
# Different regressions
lam  = 0.1
phi2 = phi[:,1:]
E1 = np.zeros(100)
E2 = np.zeros(100)
E3 = np.zeros(100)
E4 = np.zeros(100)
E5 = np.zeros(100)
E6 = np.zeros(100)

A1 = np.zeros((M,100))
A2 = np.zeros((M,100))
A3 = np.zeros((M,100))
A4 = np.zeros((M,100))
A5 = np.zeros((M,100))
A6 = np.zeros((M,100))

paths = []
for jj in range(100):
# for jj in range(10):
    f = np.power(x,2)+0.2*np.random.randn(n)
    
    a1 = np.linalg.pinv(phi) @ f
    f1 = phi @ a1
    E1[jj] = np.linalg.norm(f-f1,ord=2)/np.linalg.norm(f,ord=2)
    
    a2 = np.linalg.lstsq(phi,f,rcond=None)[0]
    f2 = phi @ a2
    E2[jj] = np.linalg.norm(f-f2,ord=2)/np.linalg.norm(f,ord=2)
    
    regr3 = linear_model.ElasticNet(alpha=1.0, copy_X=True, l1_ratio=lam, max_iter=10**5,random_state=0)
    regr3.fit(phi, f)  
    a3 = regr3.coef_
    f3 = phi @ a3
    E3[jj] = np.linalg.norm(f-f3,ord=2)/np.linalg.norm(f,ord=2)
    
    regr4 = linear_model.ElasticNet(alpha=0.8, copy_X=True, l1_ratio=lam, max_iter=10**5,random_state=0)
    regr4.fit(phi, f)  
    a4 = regr4.coef_
    f4 = phi @ a4
    E4[jj] = np.linalg.norm(f-f4,ord=2)/np.linalg.norm(f,ord=2)
    
    huber = linear_model.HuberRegressor().fit(phi, f) # matlab's robustfit() does not have an exact sklearn analogue
    a5 = huber.coef_
    f5 = phi @ a5
    E5[jj] = np.linalg.norm(f-f5,ord=2)/np.linalg.norm(f,ord=2)

    ridge = linear_model.Ridge(alpha=1.0).fit(phi,f)
    a6 = ridge.coef_
    f6 = phi @ a6
    E6[jj] = np.linalg.norm(f-f6,ord=2)/np.linalg.norm(f,ord=2)

    
    A1[:,jj] = a1
    A2[:,jj] = a2
    A3[:,jj] = a3
    A4[:,jj] = a4
    A5[:,jj] = a5
    A6[:,jj] = a6

    paths.append(f)
    print(".", end="")
print()
    
Err  = np.column_stack((E1,E2,E3,E4,E5,E6))
Err2 = np.column_stack((E1,E2,E3,E4,E5))

hv.Path((x, np.array(paths).T)).opts(width=500)

In [ ]:
def box(dat, title=''):
    df = pd.DataFrame( dat, columns=[str(i) for i in range(dat.shape[1])] )
    return df.hvplot.box().opts(title=title)

h=box(A1.T,'pinv')+box(A2.T,'lstsq')+box(A3.T,'elastic (alpha=1)')+box(A4.T,'elastic (alpha=0.8)')+box(A5.T,'huber')+box(A6.T, 'ridge')

h.opts(hv.opts.BoxWhisker(width=400)).opts(shared_axes=False).cols(2)

In [ ]:
box(Err)

In [ ]:
M   = 10
En  = np.zeros((100,M))
phi = np.zeros((len(x),M))
for jj in range(M):
    for j in range(jj):
        phi[:,j] = np.power(x,j)
    f = np.power(x,2)
    for j in range(100):
        fn       = np.power(x,2) + 0.1*np.random.randn(n)
        an       = np.linalg.pinv(phi) @ fn
        fna      = phi @ an
        En[j,jj] = np.linalg.norm(f-fna,ord=2)/np.linalg.norm(f,ord=2)  

In [ ]:
box(En)